<a href="https://colab.research.google.com/github/SwapnilAryan97/EmailSpamDetection/blob/master/EmailSpamDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!cat /proc/meminfo

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
MemTotal:       13333552 kB
MemFree:         9488396 kB
MemAvailable:   11898536 kB
Buffers:           78116 kB
Cached:          2374544 kB
SwapCached:            0 kB
Active:          1196516 kB
Inactive:        2244120 kB
Active(anon):     845220 kB
Inactive(anon):     8552 kB
Active(file):     351296 kB
Inactive(file):  2235568 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               616 kB
Writeback:             0 kB
AnonPages:        987944 kB
Mapped:           527968 kB
Shmem:              9164 kB
Slab:             174268 kB
SReclaimable:     128192 kB
SUnreclaim:        46076 kB
KernelStack:        3984 kB
PageTables:         8128 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    3178560 kB
VmallocTotal:   34359

Importing the dataset

In [2]:
url = 'https://raw.githubusercontent.com/SwapnilAryan97/EmailSpamDetection/master/emails.csv'
dataset = pd.read_csv(url)
print(dataset)
row_size = dataset.shape[0]
print(row_size)

                                                   text  spam
0     Subject: naturally irresistible your corporate...     1
1     Subject: the stock trading gunslinger  fanny i...     1
2     Subject: unbelievable new homes made easy  im ...     1
3     Subject: 4 color printing special  request add...     1
4     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5723  Subject: re : research and development charges...     0
5724  Subject: re : receipts from visit  jim ,  than...     0
5725  Subject: re : enron case study update  wow ! a...     0
5726  Subject: re : interest  david ,  please , call...     0
5727  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]
5728


Cleaning the texts

In [3]:
corpus = []
for i in range (0, row_size):
    review = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

Creating Bag of Words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = len(corpus) )
x = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

Splitting dataset into Training and Testing set

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

Kernel SVC

In [6]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf')
classifier.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Predicting Test Set result

In [7]:
y_pred = classifier.predict(x_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 1]]


Making Confusion Matrix & Accuracy Score

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy is: {:.2f}%'.format(accuracy_score(y_test, y_pred)*100))

[[860   5]
 [ 25 256]]
Accuracy is: 97.38%


Applying k-Fold Cross Validation

In [9]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
print('Accuracy: {:.2f}%'.format(accuracies.mean()*100))

Accuracy: 97.82%


Grid Search to find best model and best parameter

In [11]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[0.25, 0.5, 0.75, 1], 
              'kernel' : ['rbf']}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(x_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best Accuracy: 97.82 %
Best Parameters: {'C': 1, 'kernel': 'rbf'}
